In [8]:
import torch 
import torch.nn as nn
import numpy as np
import scipy.io
from functools import partial
from pyDOE import lhs
import time
from scipy.interpolate import griddata
import matplotlib.gridspec as gridspec
from itertools import product, combinations
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imageio
import pandas as pd
import sys
sys.path.insert(0, '../../Utilities/')
#from utils_plots import * 
from plotting import *#newfig, savefig
sys.path.insert(0, '.')
import warnings

warnings.filterwarnings("ignore")

def set_seed(seed: int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class NSNN(nn.Module):
    def __init__(self):
        super(NSNN, self).__init__()
        self.linear_in = nn.Linear(3, 20, dtype=torch.float64)
        self.linear_out = nn.Linear(20, 2, dtype=torch.float64)
        self.layers = nn.ModuleList([nn.Linear(20, 20, dtype=torch.float64) for _ in range(9)])
        self.act = nn.Tanh()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear_in(x)
        for layer in self.layers:
            x = self.act(layer(x))
        x = self.linear_out(x)
        return x     

def derivative(dy: torch.Tensor, x: torch.Tensor, order: int = 1) -> torch.Tensor:
    for i in range(order):
        dy = torch.autograd.grad(
            dy, x, grad_outputs = torch.ones_like(dy), create_graph=True, retain_graph=True
        )[0]
    return dy

def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)
        
def f(model, x_train_pt, y_train_pt, t_train_pt):
    psi_and_p = model(torch.stack((x_train_pt, y_train_pt, t_train_pt), axis = 1).view(-1, 3))
    psi = psi_and_p[:,0:1]
    p = psi_and_p[:,1:2]
    
    u = derivative(psi, y_train_pt, order=1)
    v = -derivative(psi, x_train_pt, order=1)
    
    u_t = derivative(u, t_train_pt, order=1)
    u_x = derivative(u, x_train_pt, order=1)
    u_y = derivative(u, y_train_pt, order=1)
    u_xx = derivative(u_x, x_train_pt, order=1)
    u_yy = derivative(u_y, y_train_pt, order=1)
    
    v_t = derivative(v, t_train_pt, order=1)
    v_x = derivative(v, x_train_pt, order=1)
    v_y = derivative(v, y_train_pt, order=1)
    v_xx = derivative(v_x, x_train_pt, order=1)
    v_yy = derivative(v_y, y_train_pt, order=1)    
    
    p_x = derivative(p, x_train_pt, order=1)
    p_y = derivative(p, y_train_pt, order=1)
    
    f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy)
    f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)
    
    return u, v, p, f_u, f_v

set_seed(42)
iter = 0

N_train = 5000
results = []

# Load Data
data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')
        
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

######################################################################
######################## Noiseles Data ###############################
######################################################################
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

x_train_pt = torch.from_numpy(x_train)
x_train_pt.requires_grad = True
y_train_pt = torch.from_numpy(y_train)
y_train_pt.requires_grad = True
t_train_pt = torch.from_numpy(t_train)
t_train_pt.requires_grad = True
u_train_pt = torch.from_numpy(u_train)
v_train_pt = torch.from_numpy(v_train)

lambda_1 = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
lambda_2 = torch.nn.Parameter(torch.zeros(1, requires_grad=True))    
lambda_1s = []
lambda_2s = []
images = []

iter_num=100

model_path = f'models/pt_model_NS_clean_{iter_num}.pt'
model = NSNN()
model.load_state_dict(torch.load(model_path))
model.eval()

# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]   

x_star_pt = torch.from_numpy(x_star)
x_star_pt.requires_grad = True
y_star_pt = torch.from_numpy(y_star)
y_star_pt.requires_grad = True
t_star_pt = torch.from_numpy(t_star)
t_star_pt.requires_grad = True    

u_pred, v_pred, p_pred, f_u_pred, f_v_pred = f(model, x_star_pt, y_star_pt, t_star_pt) 
u_pred = u_pred.detach().numpy()
v_pred = v_pred.detach().numpy()
p_pred = p_pred.detach().numpy()
lambda_1_value = lambda_1.detach().numpy()   
lambda_2_value = lambda_2.detach().numpy()   
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)    
error_lambda_1 = np.abs(lambda_1.detach().numpy() - 1.0)*100
error_lambda_2 = np.abs(lambda_2.detach().numpy() - 0.01)/0.01 * 100
        
print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error p: %e' % (error_p))     
print('Error l1: %.5f%%' % (error_lambda_1))                             
print('Error l2: %.5f%%' % (error_lambda_2))  



Error u: 1.018732e+00
Error v: 1.029615e+00
Error p: 1.339418e+00
Error l1: 100.00000%
Error l2: 100.00000%


In [26]:
lambda_1_values_clean = pd.read_csv('outputs/lambda_1s_clean.csv')
lambda_2_values_clean = pd.read_csv('outputs/lambda_2s_clean.csv')
lambda_1_values_noisy = pd.read_csv('outputs/lambda_1s_noisy.csv')
lambda_2_values_noisy = pd.read_csv('outputs/lambda_2s_noisy.csv')

lambda_1_value_clean=lambda_1_values_clean['l1'][iter_num-1]
lambda_2_value_clean=lambda_2_values_clean['l2'][iter_num-1]
lambda_1_value_noisy=lambda_1_values_noisy['l1'][iter_num-1]
lambda_2_value_noisy=lambda_2_values_noisy['l2'][iter_num-1]